# Make ADMIN2 malaria data (can only do it for places years in stage 2 malaria df if easier)

# Ingredients
 - stage 2 malaria df locations / years
 - population at all hierearchy from lsae to fhs
 - malaria at fhs

# Get stage 2 malaria_df
# Get the as population for those places
# Merge
# Everyone in a place / year gets the same pfpr
# Model incidence and 

In [1]:
import pandas as pd
import itertools

In [2]:
cause_map = {
    "malaria":{
        "cause_id": 345
    },
    "dengue": {
        "cause_id": 357
    }
}

measure_map = {
    "mortality": {
        "measure_id": 1,
        "name": "mortality",
        "rate_name": "Mortality rate",
        "count_name": "Deaths"
    },
    "incidence": {
        "measure_id": 6, 
        "name": "incidence",
        "rate_name": "Incidence rate",
        "count_name": "Cases"
    }
}

metric_map = {
    "rate": {
        "name": "rate",
        "metric_id": 3
    },
    "count": {
        "name": "count",
        "metric_id": 1
    },
}

age_type_map = {
    "all_ages": {
        "name": "All Ages",
        "age_type": "aa"
    },
    "age_specific": {
        "name": "Age-specific",
        "age_type": "as"
    }
}

In [3]:
PROCESSED_DATA_PATH = "/mnt/team/idd/pub/forecast-mbp/02-processed_data"
MODELING_DATA_PATH = "/mnt/team/idd/pub/forecast-mbp/03-modeling_data"
FHS_DATA_PATH = f"{PROCESSED_DATA_PATH}/age_specific_fhs"
FORECASTING_DATA_PATH = "/mnt/team/idd/pub/forecast-mbp/04-forecasting_data"

hierarchy_df_path = f"{FORECASTING_DATA_PATH}/hierarchy_lsae_1209_full.parquet"
fhs_hierarchy_df_path = f"{FHS_DATA_PATH}/fhs_hierarchy.parquet"
age_metadata_df_path = f"{FHS_DATA_PATH}/age_metadata.parquet"
fhs_data_key = "{age_type}_cause_id_{cause_id}_measure_id_{measure_id}_metric_id_{metric_id}_fhs.parquet"
look_uptable_df_path = f"{FHS_DATA_PATH}/lsae_to_fhs_table.parquet"

In [4]:
as_lsae_population_df_path = f"{MODELING_DATA_PATH}/as_lsae_population_df.parquet"
malaria_stage_2_modeling_df_path = "{MODELING_DATA_PATH}/malaria_stage_2_modeling_df.parquet"

def ensure_id_columns_are_integers(df):
    """
    Ensures that any column ending with '_id' is cast to integer type.
    """
    for col in df.columns:
        if col.endswith('_id') and pd.api.types.is_float_dtype(df[col].dtype):
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Capital I
    return df
def read_parquet_with_integer_ids(path, **kwargs):
    """Read a parquet file and ensure ID columns are integers."""
    df = pd.read_parquet(path, **kwargs)
    return ensure_id_columns_are_integers(df)

In [5]:
# Load the data
fhs_hierarchy_df = read_parquet_with_integer_ids(fhs_hierarchy_df_path)

hierarchy_df = read_parquet_with_integer_ids(hierarchy_df_path)
malaria_df = read_parquet_with_integer_ids(malaria_stage_2_modeling_df_path.format(MODELING_DATA_PATH=MODELING_DATA_PATH))

hierarchy_df = hierarchy_df[hierarchy_df["level"] >= 3].reset_index(drop=True)


look_uptable_df = read_parquet_with_integer_ids(look_uptable_df_path)

In [6]:
malaria_stage_2_modeling_df_path = "{MODELING_DATA_PATH}/malaria_stage_2_modeling_df.parquet"
malaria_df = read_parquet_with_integer_ids(malaria_stage_2_modeling_df_path.format(MODELING_DATA_PATH=MODELING_DATA_PATH))

malaria_location_ids = malaria_df["location_id"].unique().tolist()

malaria_ancestor_location_ids = look_uptable_df[look_uptable_df["location_id"].isin(malaria_location_ids)]["fhs_location_id"].unique().tolist()
all_malaria_location_ids = malaria_location_ids + malaria_ancestor_location_ids
# Get unique location IDs
all_malaria_location_ids = list(set(all_malaria_location_ids))
# Create a DataFrame with the unique location IDs
year_ids = malaria_df["year_id"].unique().tolist()


In [7]:
################################################################
#### Make filters based on malaria_df
################################################################
all_location_filter = ('location_id', 'in', all_malaria_location_ids)
fhs_location_filter = ('location_id', 'in', malaria_ancestor_location_ids)
year_filter = ('year_id', 'in', year_ids)

################################################################
#### Read LSAE population data
################################################################
as_lsae_population_df_path = f"{MODELING_DATA_PATH}/as_lsae_population_df.parquet"
# Read only the filtered rows
as_lsae_population_df = read_parquet_with_integer_ids(
    as_lsae_population_df_path,
    filters=[[all_location_filter, year_filter]]  # Combining with AND logic
)

In [8]:
################################################################
#### Load Age Metadata
################################################################
age_metadata_df = read_parquet_with_integer_ids(age_metadata_df_path)
age_group_ids = age_metadata_df["age_group_id"].unique()
sex_ids = [1, 2]  # 1
combinations = list(itertools.product(age_group_ids, sex_ids))
as_df = pd.DataFrame(combinations, columns=['age_group_id', 'sex_id'])

In [ ]:
################################################################
#### Make Age-Specific DataFrame
################################################################
as_malaria_df = malaria_df.copy()
pf_columns = 

In [42]:
as_malaria_df

,location_id,year_id,population,malaria_pfpr,parent_id,level,location_name,path_to_top_parent,A0_location_id,malaria_pf_inc_rate,...,log_malaria_pf_mort_rate,log_malaria_pf_inc_rate,log_mal_DAH_total_per_capita,log_gdppc_mean,log_ldipc_mean,logit_urban_1km_threshold_300,logit_urban_100m_threshold_300,logit_urban_1km_threshold_1500,logit_urban_100m_threshold_1500,logit_malaria_pfpr
0,25355,2000,146343.203125,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,...,-6.575129,-0.736825,-3.603867,8.432154,8.377332,6.906755,6.738462,4.786172,4.952279,0.882528
1,25355,2001,149733.796875,7.401407e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.799647e-01,...,-6.552248,-0.734041,-3.663076,8.457949,8.380703,6.906742,6.821560,4.685878,5.051461,1.042856
2,25355,2002,153356.593750,6.964961e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.760968e-01,...,-6.567546,-0.742132,-4.193325,8.588520,8.419079,6.906719,6.871901,4.584427,5.204650,0.827375
3,25355,2003,157241.343750,6.727383e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.724381e-01,...,-6.561327,-0.749846,-3.902903,8.673434,8.475221,6.906687,6.906687,4.582583,5.295903,0.717542
4,25355,2004,161380.968750,6.371068e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.640392e-01,...,-6.591201,-0.767784,-2.623802,8.787557,8.555652,6.906792,6.872414,4.492308,5.396668,0.560074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198357,95624,2004,9750.993164,8.167190e-09,4772,3,Baro,"1,135,4772,95624",135,2.136536e-08,...,-13.815445,-13.794371,-13.815511,9.186859,9.138649,-0.921136,1.464642,-inf,-1.593432,-18.624142
198358,95624,2005,9847.751953,8.703803e-09,4772,3,Baro,"1,135,4772,95624",135,2.276915e-08,...,-13.815454,-13.792996,-4.191712,9.208657,9.153932,-0.912316,1.430417,-inf,-1.564319,-18.560507
198359,95624,2006,9910.347656,3.351024e-08,4772,3,Baro,"1,135,4772,95624",135,8.766277e-08,...,-13.815384,-13.731480,-5.858559,9.253010,9.175848,-0.895598,1.436006,-inf,-1.557979,-17.212416
198360,95624,2007,9966.488281,9.968910e-08,4772,3,Baro,"1,135,4772,95624",135,2.607866e-07,...,-13.815261,-13.583775,-3.432458,9.314377,9.206165,-0.734150,1.445928,-inf,-1.509345,-16.122210


In [15]:
################################################################
#### Make Age-Specific DataFrame
################################################################
as_malaria_df = malaria_df.copy()
as_malaria_df = as_malaria_df.drop(columns=['population'])
as_malaria_df = as_malaria_df.merge(as_df, how = "cross")
as_malaria_df = as_malaria_df.merge(as_lsae_population_df, on=["location_id", "year_id","age_group_id", "sex_id"], how="left")

In [34]:
################################################################
#### Get FHS Data for Malaria
################################################################
fhs_data_path = "{FHS_DATA_PATH}/{age_type}_cause_id_{cause_id}_measure_id_{measure_id}_metric_id_{metric_id}_fhs.parquet"

cause = "malaria"
metric = "count"

cause_id = cause_map[cause]["cause_id"]
age_type = age_type_map["age_specific"]["age_type"]
metric_id = metric_map[metric]["metric_id"]

fhs_outcome_count_dfs = {}
for measure in measure_map:
    measure_id = measure_map[measure]["measure_id"]
    fhs_data_path_full = fhs_data_path.format(
        FHS_DATA_PATH=FHS_DATA_PATH,
        age_type=age_type,
        cause_id=cause_id,
        measure_id=measure_id,
        metric_id=metric_id
    )
    
    df = read_parquet_with_integer_ids(fhs_data_path_full,
        filters=[[fhs_location_filter, year_filter]])
    
    df = df.rename(columns={
        "population": "fhs_population",
        "location_id": "fhs_location_id"
    })
    fhs_outcome_count_dfs[measure] = df

In [39]:
measure = "mortality"
df = fhs_outcome_count_dfs["mortality"]
df = df[["fhs_location_id", "year_id", "age_group_id","sex_id","val","fhs_population"]]
as_malaria_df = as_malaria_df.merge(df, on=["fhs_location_id", "year_id", "age_group_id","sex_id"], how="left")

In [40]:
as_malaria_df

,location_id,year_id,malaria_pfpr,parent_id,level,location_name,path_to_top_parent,A0_location_id,malaria_pf_inc_rate,malaria_pf_inc_count,...,logit_urban_100m_threshold_1500,logit_malaria_pfpr,age_group_id,sex_id,population_aa,fhs_location_id,pop_fraction_aa,population,val,fhs_population
0,25355,2000,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,70044.289062,...,4.952279,0.882528,2,1,146343.203125,25318,0.000101,14.795592,0.000000,3.499641e+03
1,25355,2000,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,70044.289062,...,4.952279,0.882528,2,2,146343.203125,25318,0.000124,18.201478,0.000000,4.305244e+03
2,25355,2000,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,70044.289062,...,4.952279,0.882528,3,1,146343.203125,25318,0.000510,74.698269,51.082252,1.766858e+04
3,25355,2000,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,70044.289062,...,4.952279,0.882528,3,2,146343.203125,25318,0.012984,1900.058476,28.676127,4.494259e+05
4,25355,2000,7.080539e-01,25318,3,Aba North Local Government Area,"1,214,25318,25355",214,4.786303e-01,70044.289062,...,4.952279,0.882528,6,1,146343.203125,25318,0.000003,0.371247,152.283259,8.781199e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10263145,95624,2008,1.714921e-07,4772,3,Baro,"1,135,4772,95624",135,4.486230e-07,0.004494,...,-1.450562,-15.579729,238,2,10017.603516,4772,0.001323,13.252395,0.000197,4.901935e+04
10263146,95624,2008,1.714921e-07,4772,3,Baro,"1,135,4772,95624",135,4.486230e-07,0.004494,...,-1.450562,-15.579729,388,1,10017.603516,4772,0.000171,1.711548,0.000112,6.330853e+03
10263147,95624,2008,1.714921e-07,4772,3,Baro,"1,135,4772,95624",135,4.486230e-07,0.004494,...,-1.450562,-15.579729,388,2,10017.603516,4772,0.043718,437.947457,0.000141,1.619926e+06
10263148,95624,2008,1.714921e-07,4772,3,Baro,"1,135,4772,95624",135,4.486230e-07,0.004494,...,-1.450562,-15.579729,389,1,10017.603516,4772,0.011192,112.115111,0.000075,4.147031e+05


In [30]:
for measure, df in fhs_outcome_count_dfs.items():
    as_malaria_df = as_malaria_df.merge(df, on=["fhs_location_id", "year_id", "age_group_id","sex_id","val","population"], how="left")
    as_malaria_df = as_malaria_df.rename(columns={"val": f"{cause}_{measure}_{metric}_fhs"})

KeyError: 'val'

In [27]:
fhs_outcome_count_dfs["mortality"]

,age_group_id,fhs_location_id,year_id,sex_id,cause_id,measure_id,metric_id,malaria_mortality_count_fhs,upper,lower,...,parent_id,path_to_top_parent,level,most_detailed,location_name,super_region_id,super_region_name,region_id,region_name,age_group_name
0,2,200,2020,1,345,1,1,0.000000,0.000000,0.000000,...,199,"1,166,199,200",3,1,Benin,166,Sub-Saharan Africa,199,Western Sub-Saharan Africa,Early Neonatal
1,2,4756,2016,1,345,1,1,0.000000,0.000000,0.000000,...,135,"1,103,134,135,4756",4,1,Distrito Federal,103,Latin America and Caribbean,134,Tropical Latin America,Early Neonatal
2,2,4874,2021,1,345,1,1,0.000000,0.000000,0.000000,...,163,"1,158,159,163,4874",4,1,Uttarakhand,158,South Asia,159,South Asia,Early Neonatal
3,2,4874,2021,2,345,1,1,0.000000,0.000000,0.000000,...,163,"1,158,159,163,4874",4,1,Uttarakhand,158,South Asia,159,South Asia,Early Neonatal
4,2,4709,2016,1,345,1,1,0.000000,0.000000,0.000000,...,11,"1,4,9,11,4709",4,1,Aceh,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,Early Neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219645,389,169,2012,2,345,1,1,397.010007,790.955863,162.821148,...,167,"1,166,167,169",3,1,Central African Republic,166,Sub-Saharan Africa,167,Central Sub-Saharan Africa,6-11 months
219646,389,4720,2008,1,345,1,1,0.001112,0.003881,0.000001,...,11,"1,4,9,11,4720",4,1,Jakarta,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,6-11 months
219647,389,4720,2008,2,345,1,1,0.001721,0.006502,0.000002,...,11,"1,4,9,11,4720",4,1,Jakarta,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,6-11 months
219648,389,4775,2013,2,345,1,1,0.002277,0.004332,0.001102,...,135,"1,103,134,135,4775",4,1,São Paulo,103,Latin America and Caribbean,134,Tropical Latin America,6-11 months


In [ ]:
fhs_outcome_count_df[["age_group_id", "fhs_location_id", "year_id", "sex_id", ]]

In [21]:
fhs_outcome_count_df

,age_group_id,location_id,year_id,sex_id,cause_id,measure_id,metric_id,val,upper,lower,...,parent_id,path_to_top_parent,level,most_detailed,location_name,super_region_id,super_region_name,region_id,region_name,age_group_name
0,2,200,2020,1,345,1,1,0.000000,0.000000,0.000000,...,199,"1,166,199,200",3,1,Benin,166,Sub-Saharan Africa,199,Western Sub-Saharan Africa,Early Neonatal
1,2,4756,2016,1,345,1,1,0.000000,0.000000,0.000000,...,135,"1,103,134,135,4756",4,1,Distrito Federal,103,Latin America and Caribbean,134,Tropical Latin America,Early Neonatal
2,2,4874,2021,1,345,1,1,0.000000,0.000000,0.000000,...,163,"1,158,159,163,4874",4,1,Uttarakhand,158,South Asia,159,South Asia,Early Neonatal
3,2,4874,2021,2,345,1,1,0.000000,0.000000,0.000000,...,163,"1,158,159,163,4874",4,1,Uttarakhand,158,South Asia,159,South Asia,Early Neonatal
4,2,4709,2016,1,345,1,1,0.000000,0.000000,0.000000,...,11,"1,4,9,11,4709",4,1,Aceh,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,Early Neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439295,389,169,2012,2,345,6,1,49357.123866,73248.949063,23555.474563,...,167,"1,166,167,169",3,1,Central African Republic,166,Sub-Saharan Africa,167,Central Sub-Saharan Africa,6-11 months
439296,389,4720,2008,1,345,6,1,3.755468,18.997297,0.001051,...,11,"1,4,9,11,4720",4,1,Jakarta,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,6-11 months
439297,389,4720,2008,2,345,6,1,3.529011,17.851745,0.000988,...,11,"1,4,9,11,4720",4,1,Jakarta,4,"Southeast Asia, East Asia, and Oceania",9,Southeast Asia,6-11 months
439298,389,4775,2013,2,345,6,1,0.285516,0.383850,0.210589,...,135,"1,103,134,135,4775",4,1,São Paulo,103,Latin America and Caribbean,134,Tropical Latin America,6-11 months
